In [508]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import os
import uuid

In [592]:
#etl = 'kobo2elastic'
#etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'
etl = 'cuartero2newaqm'
#etl = 'cambodia2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

root_object = 'resident'

if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/koboSchema.1-item.json'
    input_data_file = 'data/koboData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/koboSchema/'
    
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-item.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/KHMSchema.json'
    input_data_file = 'data/KHM.3-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/KHMSchema/'

elif etl == 'cuartero2newaqm':
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    #input_data_file = 'data/Isabela.2-items.json'
    #mapping_file = 'schema/map/Philippines/2.2.AQMHealthInfoQuestionsV1.csv'
    #mapping_file = 'schema/map/Philippines/2.2.AQMHealthInfoQuestionsV1.csv'
    mapping_file = 'schema/map/Philippines/2.1.AQMGeneralQuestionsV1.csv'
    datelog_dir = 'couchbase-curis-2019-06-21-cuartero'
    processed_dir    = 'data/processed/' + datelog_dir + '/'
    tmp_dir    = 'data/processed/' + datelog_dir + '/tmp/'
    output_dir = 'data/processed/' + datelog_dir + '/'
    merged_dir = 'data/merged/' + datelog_dir + '/'
    


## TODO: CREATE FOLDER DIRECTORY MKDIR
## TODO: PARSED FIELD TYPE: LIST -- merged into single object

## TODO: FIX ID MERGING

## TODO: FIX _id into profileId

# READ MAPPING FILE

In [604]:
#def _get_mapping_fields():
_mapping_df = pd.DataFrame()
_mapping_df = read_csv(mapping_file).sort_values(['source_key']).replace(np.nan,'',regex=True)
_mapping_df

,source_key,source_type,destination_key,destination_key.1,data_type,data_source,data_format,default_value,lookup_value
0,profiles.civil_status,string,answers.civilStatus,string,existing,original,,,
2,profiles.education,string,answers.highestEducation,string,existing,original,,,
4,profiles.employment.is_employed,string,answers.employmentStatus,string,existing,original,,,
3,profiles.employment.nature,string,answers.employmentArea,string,existing,original,,,
1,profiles.religion,string,answers.religion,string,existing,original,,,
5,,,dateUpdated,timestamp,new,generated,timestamp,,
6,,,dateCreated,timestamp,new,generated,timestamp,2019-06-27T20:36:43.993+08:00,
7,,,formName,string,new,generated,,2.1 - AQM General Questions V1,
8,,,formId,string,new,generated,,AWviKwKDKxqtHWZMp2v4,
9,,,type,string,new,generated,,profile-related-form,


# GET REQUIRED FIELDS IN THE MAPPING

In [558]:
mapping_fields_list = []
mapping_fields_list = list(_mapping_df['source_key'].unique())
mapping_fields_list

['profiles.civil_status',
 'profiles.education',
 'profiles.employment.is_employed',
 'profiles.employment.nature',
 'profiles.religion',
 nan]

# GET FIELD NAMES OF THE SCHEMA using _META

In [633]:
meta_headers_df = pd.DataFrame()
meta_headers_df = pd.read_csv( 'data/processed/couchbase-curis-2019-06-21/_meta.csv')
meta_headers_df.head(3)

,file_name,field_name,field_type
0,resident,middle_name,primitive
1,resident,birthdate,primitive
2,resident,id,primitive


In [632]:
match_headers_df = pd.DataFrame()
match_headers_df = meta_headers_df[meta_headers_df['field_name'].isin(mapping_fields_list)]
match_headers_df = match_headers_df.sort_values(['file_name','field_name']).reset_index(drop=True)
match_headers_df.head(3)

,file_name,field_name,field_type
0,profiles,profiles.civil_status,primitive
1,profiles,profiles.education,primitive
2,profiles,profiles.employment.is_employed,primitive


# CREATE DEFAULT DICT FOR FILENAME AS KEY and FIELD NAMES AS VALUE

In [580]:
filename_per_field_dd = defaultdict(list)

for index,row in match_headers_df.iterrows():
    filename = row['file_name']
    fields = row['field_name']
    filename_per_field_dd[filename].append(fields)  
    
filename_per_field_dd

defaultdict(list,
            {'profiles': ['profiles.civil_status',
              'profiles.education',
              'profiles.employment.is_employed',
              'profiles.employment.nature',
              'profiles.religion']})

## REMOVE FILE IF EXISTING

In [596]:
os.remove(merged_dir + 'general_questions'+ '.csv', )

## WRITE EMPTY CSV FOR MERGE.csv

### TODO: FILENAME MUST BE DYNAMIC

In [597]:
test_filename_per_field_dd = {
    'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
    'health_informations.family_history': ['health_informations.family_history']}

test_filename_per_field_dd = {'profiles': ['profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']}

fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
all_fields_list  = ['_id','_index_map'] + flat_fields_list

empty_data_df = pd.DataFrame(columns=all_fields_list)
empty_data_df.to_csv(merged_dir + 'general_questions'+ '.csv', encoding='utf-8', mode='w', header=True,index=False)

In [281]:
test_df = pd.DataFrame()
for filename, fields in test_filename_per_field_dd.items():
    print('\n')
    print('---FILENAME: ', filename)
    print('---FIELDS: ',fields)

    
    _individual_data_df = pd.DataFrame()
    _individual_data_df = pd.read_csv(processed_dir + filename + '.csv').sort_values(['_id','_index_map'])  
    
    filename_fields_list = ['_id','_index_map'] + fields
    _individual_data_df = _individual_data_df[filename_fields_list]
    
    _required_data_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv',nrows=0)
    
    for index, row in _individual_data_df.iterrows():

        #_unique_id_index_df = pd.read_csv(merged_dir + 'health_informations'+ '.csv', dtype={'_index_map': str})[['_id','_index_map']].replace(np.nan,'',regex=True)
        #_uniqued_ids_list = list(_unique_id_index_df['_id'].unique())
        #_existing_id = ''
        
        for header in row.index:
            
            data_value = row[header]
            print('HEADER :' + header + '  value: ' + str(value) )
            
            
            if header == '_id':
                if data_value in _uniqued_ids_list:
                    _existing_id = unique_id_index_df[unique_id_index_df['_id'] == data_value].index[0]
                    
                    
                    print('existing at id: ', data_value)
                    print(_existing_id)
                    _required_data_df.at['',header] = row[header]
                else:
                    print('not existing')
                    _required_data_df.at['',header] = row[header]
            else:
                _required_data_df.at['',header] = row[header]
        print('------')
        
        
        _required_data_df.to_csv(merged_dir + 'health_informations'+ '.csv', encoding='utf-8', mode='a',header=False,index=False)




---FILENAME:  resident
---FIELDS:  ['gender', 'birthdate']
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  2f920d39-319e-4bf7-90c8-6133c3337af7
0
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :gender  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :birthdate  value: 01/01/2009 at 08:44:13 PM GMT+08:00
------
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  64c2e7de-0cc8-4df5-a54f-02398652c080
1
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :gender  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :birthdate  value: 01/01/2009 at 08:44:13 PM GMT+08:00
------


---FILENAME:  profiles
---FIELDS:  ['profiles.employment.monthly_income']
HEADER :_id  value: 01/01/2009 at 08:44:13 PM GMT+08:00
existing at id:  2f920d39-319e-4bf7-90c8-6133c3337af7
0
HEADER :_index_map  value: 01/01/2009 at 08:44:13 PM GMT+08:00
HEADER :profiles.employment.monthly_income  value: 01/01/2009 at 08:44:13 PM GMT+0

## ==============DUMMY GET DATA===========

## FAMILY PLANNING

In [618]:
_fp_df = pd.read_csv(processed_dir + 'family_planning_and_maternal_healths' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_fp_df = _fp_df[['_id','_index_map',
              'family_planning_and_maternal_healths.are_you_pregnant',
              'family_planning_and_maternal_healths.date_updated']].replace(np.nan,'',regex=True)
_fp_df.head(3)

,_id,_index_map,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,False,01/26/2019 at 07:31:27 PM GMT+08:00
1,001296cb-ffd9-4eed-8dd5-3326d0468d79,0,False,10/26/2018 at 11:00:58 AM GMT+08:00
2,00256005-2b4a-480a-a777-d208ad640183,0,False,07/26/2018 at 10:17:50 AM GMT+08:00


## RESIDENT DATA

In [630]:
_resident_df = pd.read_csv(processed_dir + 'resident' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_resident_df = _resident_df[['_id','_index_map','gender','birthdate']].replace(np.nan,'',regex=True)
_resident_df.head(3)

,_id,_index_map,gender,birthdate
0,0003ff38-28fb-4005-9437-d276cbb9da4d,,Male,04/01/1997
1,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Babae,04/12/1998
2,0006de3c-57ea-46d2-89b8-e606533d3d15,,Male,07/25/1982


## PROFILE DATA

In [617]:
_profile_df = pd.read_csv(processed_dir + 'profiles' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_profile_df = _profile_df[['_id','_index_map','profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']].replace(np.nan,'',regex=True)
_profile_df = _profile_df.replace(np.nan,'',regex=True).reset_index(drop=True)
_profile_df.head(3)

,_id,_index_map,profiles.civil_status,profiles.employment.is_employed,profiles.education,profiles.employment.nature,profiles.religion
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,Single,,No Formal Education,empty,Catholic
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,Single,,No Formal Education,empty,Catholic
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,Single,,Elementary Level,empty,Catholic


## HEALTH INFORATION DATA

In [629]:
_hi_df = pd.read_csv(processed_dir + 'health_informations' + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
_hi_df  = _hi_df [['_id','_index_map',
              'health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit']].replace(np.nan,'',regex=True).reset_index(drop=True)
_hi_df.head(3)

,_id,_index_map,health_informations.allergies,health_informations.blood_pressure.first_reading.diastole,health_informations.blood_pressure.first_reading.systole,health_informations.blood_pressure.second_reading.diastole,health_informations.blood_pressure.second_reading.systole,health_informations.blood_sign,health_informations.blood_type,health_informations.exercise_in_a_week,health_informations.smoking_habit
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,,,,,,empty,I don't know,0 to 1 hour,Never
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,,,,,,empty,I don't know,0 to 1 hour,Never
2,0003ff38-28fb-4005-9437-d276cbb9da4d,2,,,,,,empty,I don't know,0 to 1 hour,Never


## MERGE SELECTED FILES --sample script for merging dataframe

In [432]:
_merge_df = _resident_df.merge(_profile_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_hi_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df = _merge_df.merge(_fp_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') )
_merge_df.to_csv(merged_dir + 'merge' + '.csv',index=False)
_merge_df

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/1975,NaN,NaN,NaN,NaN,NaN
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,08/09/1989,NaN,NaN,NaN,NaN,NaN
2,2f920d39-319e-4bf7-90c8-6133c3337af7,0,NaN,NaN,Php 13.00,Pssive,B,No,01/01/2009 at 08:44:13 PM GMT+08:00
3,64c2e7de-0cc8-4df5-a54f-02398652c080,0,NaN,NaN,Php12.00,Active,A,No,01/01/2009 at 08:44:13 PM GMT+08:00
4,2f920d39-319e-4bf7-90c8-6133c3337af7,1,NaN,NaN,NaN,Never,,Yes,03/02/2008 at 08:44:13 PM GMT+08:00


## WRITE MERGE DATA TO CSV

In [462]:
test_group_df = pd.read_csv(merged_dir + 'merge' + '.csv',dtype={'_index_map': str}).replace(np.nan,'',regex=True)

In [621]:
test_group_df.head(3)

,_id,_index_map,gender,birthdate,profiles.employment.monthly_income,health_informations.smoking_habit,health_informations.blood_type,family_planning_and_maternal_healths.are_you_pregnant,family_planning_and_maternal_healths.date_updated
0,2f920d39-319e-4bf7-90c8-6133c3337af7,,Male,12/23/75,,,,,
1,64c2e7de-0cc8-4df5-a54f-02398652c080,,Female,8/9/89,,,,,
2,2f920d39-319e-4bf7-90c8-6133c3337af7,,,,Php 13.00,Pssive,B,No,01/01/2009 at 08:44:13 PM GMT+08:00


## RENAME THE HEADERS USING MAPPED HEADERS

In [599]:
source_destination_keys_df = pd.DataFrame()
source_destination_keys_df = _mapping_df[['source_key','destination_key']]

In [616]:
new_column_name_dict = dict(zip(source_destination_keys_df['source_key'], source_destination_keys_df['destination_key']))
new_column_name_dict 

{'profiles.civil_status': 'answers.civilStatus',
 'profiles.education': 'answers.highestEducation',
 'profiles.employment.is_employed': 'answers.employmentStatus',
 'profiles.employment.nature': 'answers.employmentArea',
 'profiles.religion': 'answers.religion',
 nan: 'type'}

## GET THE DATA from merged dataframes

In [623]:
_required_data_df = pd.DataFrame()
_required_data_df = _profile_df

## DO NECESSARY TRANSFORMATION

In [624]:
_required_data_df.rename(columns = new_column_name_dict,inplace=True )

## APPLICABLE only if not Resident data
_required_data_df.rename(columns={'_id': 'profileId'}, inplace=True)
_required_data_df.head(3)

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,Single,,No Formal Education,empty,Catholic
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,Single,,No Formal Education,empty,Catholic
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,Single,,Elementary Level,empty,Catholic


## GENERATE _id column

In [625]:
_required_data_df['_id'] = _required_data_df.index.to_series().map(lambda x: uuid.uuid4())
_required_data_df.head(3)

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion,_id
0,0003ff38-28fb-4005-9437-d276cbb9da4d,0,Single,,No Formal Education,empty,Catholic,1e92b523-19a9-45b5-ae0c-06b885a032d2
1,0003ff38-28fb-4005-9437-d276cbb9da4d,1,Single,,No Formal Education,empty,Catholic,5bd32bc1-2067-4edc-bb03-3acb3386c30b
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0,Single,,Elementary Level,empty,Catholic,415607ea-40a7-4700-a172-dceaa0144d88


## CLEAR _INDEX_MAP since each _id has generaed ID

In [626]:
_required_data_df['_index_map'] = ''
_required_data_df.head(3)

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion,_id
0,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,1e92b523-19a9-45b5-ae0c-06b885a032d2
1,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,5bd32bc1-2067-4edc-bb03-3acb3386c30b
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Elementary Level,empty,Catholic,415607ea-40a7-4700-a172-dceaa0144d88


## GET THE NEW FIELDS WITH THE DEFAULT VALUES

In [605]:
new_fields_df = _mapping_df[_mapping_df['data_type'] == 'new'][['destination_key', 'default_value']]
new_fields_df

,destination_key,default_value
5,dateUpdated,
6,dateCreated,2019-06-27T20:36:43.993+08:00
7,formName,2.1 - AQM General Questions V1
8,formId,AWviKwKDKxqtHWZMp2v4
9,type,profile-related-form


## APPEND THE NEW FIELDS AS COLUMN

In [628]:
for index,row in new_fields_df.iterrows():
    _header = row['destination_key']
    _value = row['default_value']
    _required_data_df[_header] = _value

_required_data_df = _required_data_df.reset_index(drop=True)
_required_data_df.head(5)

,profileId,_index_map,answers.civilStatus,answers.employmentStatus,answers.highestEducation,answers.employmentArea,answers.religion,_id,dateUpdated,dateCreated,formName,formId,type
0,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,1e92b523-19a9-45b5-ae0c-06b885a032d2,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
1,0003ff38-28fb-4005-9437-d276cbb9da4d,,Single,,No Formal Education,empty,Catholic,5bd32bc1-2067-4edc-bb03-3acb3386c30b,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
2,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Elementary Level,empty,Catholic,415607ea-40a7-4700-a172-dceaa0144d88,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
3,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,9b069526-c0cb-4a37-a0dc-e0da7bee6e0f,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form
4,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,Single,,Wala pormal na Edukasyon,empty,Catholic,17652c83-d0e0-46dc-9afa-f13ddbea53ad,,2019-06-27T20:36:43.993+08:00,2.1 - AQM General Questions V1,AWviKwKDKxqtHWZMp2v4,profile-related-form


# WRITE TO CSV FINALIZED


In [608]:
_required_data_df.to_csv(merged_dir + 'general_questions.csv', encoding='utf-8', mode='w', header=True,index=False)